In [1]:
import time
from pathlib import Path
from datetime import datetime
import dask.dataframe as dd
from distributed import LocalCluster, Client
import spatialpandas as spd

from spatialpandas.io import read_parquet, read_parquet_dask
import geopandas
import pygeos
import dask_geopandas

In [2]:
# set up data paths
base_path = Path().cwd().parent
data_dir = base_path.joinpath('data')

In [3]:
# create local dask cluster
cluster = LocalCluster(#silence_logs=logging.ERROR,
                       dashboard_address=':8790',
                       n_workers=1,
                       threads_per_worker=4,
                       memory_limit='4 GB')


client = Client(cluster)
client

/home/joris/miniconda3/envs/geo/lib/python3.9/site-packages/distributed/node.py:160: UserWarning: Port 8790 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 34355 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:34355/status,
Dashboard: http://127.0.0.1:34355/status,Workers: 1
Total threads: 4,Total memory: 3.73 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45227,Workers: 1
Dashboard: http://127.0.0.1:34355/status,Total threads: 4
Started: Just now,Total memory: 3.73 GiB
Comm: tcp://192.168.0.118:40233,Total threads: 4
Dashboard: http://192.168.0.118:36621/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:40591,


## Converting the spatialpandas Parquet dataset to GeoPandas Parquet dataset

In [4]:
spatial_sort_path = data_dir.joinpath('us_cont_spatiallysorted.parquet')

In [5]:
from spatialpandas.io import read_parquet, read_parquet_dask

In [6]:
df = read_parquet(spatial_sort_path / "part.0.parquet")
df

,position,latitude,longitude
hilbert_distance,,,
25629,"Point([-124.443, 24.447])",24.447000,-124.443000
99850,"Point([-124.532, 24.704])",24.704000,-124.532000
216826,"Point([-124.248, 24.591])",24.591000,-124.248000
253418,"Point([-124.085, 24.463])",24.463000,-124.085000
290728,"Point([-123.719, 24.554])",24.554000,-123.719000
...,...,...,...
51981088,"Point([-111.4537759, 26.0588929])",26.058893,-111.453776
51981088,"Point([-111.4547769, 26.0591679])",26.059168,-111.454777
51981088,"Point([-111.4536229, 26.0588199])",26.058820,-111.453623


In [7]:
def spatialpandas_to_geopandas(df):
    arr = pygeos.points(df.position.array.flat_values.reshape(-1, 2))
    gdf = geopandas.GeoDataFrame(df[["latitude", "longitude"]], geometry=arr, crs="EPSG:4326")
    return gdf

In [8]:
# load spatially sorted us data
spatial_sort_path = data_dir.joinpath('us_cont_spatiallysorted.parquet')
ddf = read_parquet_dask(spatial_sort_path)
ddf.head(2)

,position,latitude,longitude
hilbert_distance,,,
25629,"Point([-124.443, 24.447])",24.447,-124.443
99850,"Point([-124.532, 24.704])",24.704,-124.532


In [9]:
gddf = ddf.map_partitions(spatialpandas_to_geopandas)
gddf

,latitude,longitude,geometry
npartitions=1218,,,
,float64,float64,geometry
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [12]:
gddf.head(2)

,latitude,longitude,geometry
hilbert_distance,,,
25629,24.447,-124.443,POINT (-124.44300 24.44700)
99850,24.704,-124.532,POINT (-124.53200 24.70400)


In [13]:
t0 = time.time()
# spatially sort the data
savepath = data_dir.joinpath('us_cont_spatiallysorted_geopandas.parquet')
gddf.to_parquet(savepath)
dt = time.time() - t0

In [14]:
# save timing info
with open(f'to_dask_geopandas_time-{datetime.now()}.csv', 'w') as f:
    f.write(f'time_min,npartitions\n{dt/60},{gddf.npartitions}')
print('dt (s):', dt)

dt (s): 404.1104176044464


In [15]:
# check the saved file has the same data as the original
df = dask_geopandas.read_parquet(savepath)
display(df.head(2))
len(df)

,latitude,longitude,geometry
hilbert_distance,,,
25629,24.447,-124.443,POINT (-124.44300 24.44700)
99850,24.704,-124.532,POINT (-124.53200 24.70400)


113944489

In [17]:
# release the dask workers
client.shutdown()

## Repartition the GeoPandas Parquet dataset into fewer, larger partitions

In [4]:
data_path = data_dir.joinpath('us_cont_spatiallysorted_geopandas.parquet')

In [5]:
df = dask_geopandas.read_parquet(data_path)
df

,latitude,longitude,geometry
npartitions=1218,,,
25629,float64,float64,geometry
51981109,...,...,...
...,...,...,...
1027558833,...,...,...
1073720176,...,...,...


In [6]:
df2 = df.repartition(npartitions=100)

In [8]:
savepath = data_dir.joinpath('us_cont_spatiallysorted_geopandas_100.parquet')

In [9]:
df2.to_parquet(savepath)

In [10]:
# release the dask workers
client.shutdown()

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError
